# Field Phasing and Scaling (Autophase)

In [ ]:
from pmd_beamphysics import FieldMesh

import matplotlib.pyplot as plt
import numpy as np

# Get field

In [ ]:
FM = FieldMesh("../data/rfgun.h5")
FM.plot(aspect="equal", figsize=(12, 8))

In [ ]:
# On-axis field
z0 = FM.coord_vec("z")
Ez0 = FM.Ez[0, 0, :]  # this is complex
plt.plot(z0, np.real(Ez0))

# v=c voltage and phase

In [ ]:
from pmd_beamphysics.fields.analysis import accelerating_voltage_and_phase

In [ ]:
?accelerating_voltage_and_phase

In [ ]:
V0, phase0 = accelerating_voltage_and_phase(z0, -Ez0 * 120e6, FM.frequency)

V0, (phase0 * 180 / np.pi) % 360

# Tracking

Equations of motion:

$\frac{dz}{dt} = \frac{pc}{\sqrt{(pc)^2 + m^2 c^4)}} c$ 

$\frac{dp}{dt} = q E_z $

$E_z = \Re f(z) \exp(-i \omega t) $

In [ ]:
from pmd_beamphysics.fields.analysis import track_field_1d
from pmd_beamphysics.units import mec2

In [ ]:
?track_field_1d

In [ ]:
Z = FM.coord_vec("z")
E = FM.Ez[0, 0, :] * np.exp(1j * 2 * np.pi / 360 * 0) * 120e6

# Final z (m) and pz (eV/c)
track_field_1d(Z, E, FM.frequency, pz0=0, t0=0)

In [ ]:
# Use debug mode to see the actual track
sol = track_field_1d(
    Z, E, FM.frequency, pz0=0, t0=0, debug=True, max_step=1 / FM.frequency / 100
)

In [ ]:
# Plot the track
fig, ax = plt.subplots()

ax2 = ax.twinx()

ax.set_xlabel("f*t")
ax.set_ylabel("z (m)")
ax2.set_ylabel("KE (MeV)")

ax.plot(sol.t * FM.frequency, sol.y[0])
ax2.plot(sol.t * FM.frequency, (np.hypot(sol.y[1], mec2) - mec2) / 1e6, color="red")

# Autophase

In [ ]:
from pmd_beamphysics.fields.analysis import autophase_field

In [ ]:
phase_deg1, pz1 = autophase_field(FM, pz0=0, scale=120e6, verbose=True)
phase_deg1, pz1

In [ ]:
# Use debug mode to visualize. This returns the phasiing function
phase_f = autophase_field(FM, pz0=0, scale=120e6, debug=True)
phase_f(304.3348289439232)

In [ ]:
plist = np.linspace(280, 330, 100)
pzlist = np.array([phase_f(p) for p in plist])

plt.plot(plist, pzlist / 1e6)
plt.scatter(phase_deg1, pz1 / 1e6, color="red")
plt.xlabel("phase (deg)")
plt.ylabel("pz (MeV/c)")

# Autophase and Scale

In [ ]:
from pmd_beamphysics.fields.analysis import autophase_and_scale_field

?autophase_and_scale_field

In [ ]:
phase_deg2, scale2 = autophase_and_scale_field(FM, 6e6, pz0=0, verbose=True)
phase_deg2, scale2

In [ ]:
# Use debug mode to visualize. This returns the phasing function
ps_f = autophase_and_scale_field(FM, 6e6, pz0=0, debug=True)
ps_f(phase_deg2, scale2)

In [ ]:
plist = np.linspace(280, 330, 100)
denergy = np.array([ps_f(p, scale2) for p in plist])

plt.plot(plist, denergy / 1e6)
plt.scatter(phase_deg2, ps_f(phase_deg2, scale2) / 1e6, color="red", label="Autophased")
plt.xlabel("phase (deg)")
plt.ylabel("Voltage (MV)")
plt.legend()